In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install stanza --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 29.2 MB/s eta 0:00:00


In [14]:
import pandas as pd
import stanza
from typing import List, Dict

In [4]:
%cd /content/drive/MyDrive/testa_di_Marianna/dati_wiki_ristretti/database/

/content/drive/MyDrive/testa_di_Marianna/dati_wiki_ristretti/database


In [25]:
luoghi_ = pd.read_csv('napoli_particolari_luoghi.tsv', sep='\t')
luoghi_.rename(columns={'link': 'url'}, inplace=True)



In [28]:
st_= pd.read_csv('napoli_particolari_storia.tsv', sep='\t').to_dict(orient='records')
luoghi_ = pd.read_csv('napoli_particolari_luoghi.tsv', sep='\t')
luoghi_new = luoghi_.rename(columns={'link': 'url'}, inplace=False).to_dict(orient='records')
legg_ = pd.read_csv('napoli_particolari_leggende.tsv', sep='\t').to_dict(orient='records')
wiki_ = pd.read_csv('wiki_naples_chunks_filtered.tsv', sep='\t').to_dict(orient='records')

In [29]:
all_data = st_ + luoghi_new + legg_ + wiki_

In [12]:
type(all_data)

list

In [13]:

stanza.download('it')
nlp = stanza.Pipeline(lang='it', processors='tokenize')


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: it (Italian) ...
INFO:stanza:File exists: /root/stanza_resources/it/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Done loading processors!


In [32]:
def chunk_text_using_stanza(data_list: List[Dict[str, str]], max_tokens: int = 512, overlap: int = 10) -> List[Dict[str, str]]:

    chunks = []

    for data in data_list:
      try:
        title = data["title"]
        text = data["text"]
        url = data["url"]

        # Applica Stanza sul testo
        doc = nlp(text)
        sentences = [sent.text for sent in doc.sentences]

        current_chunk = []
        current_token_count = 0

        for sentence in sentences:
            sentence_tokens = sentence.split()  # Usa la divisione per spazi per ottenere i token
            num_tokens_in_sentence = len(sentence_tokens)

            if num_tokens_in_sentence > max_tokens:
                # Gestisce le frasi più lunghe di max_tokens
                start = 0
                end = max_tokens
                while start < num_tokens_in_sentence:
                    chunk = " ".join(sentence_tokens[start:end])
                    chunks.append({"title": title, "text": chunk, "url": url})
                    start += max_tokens - overlap
                    end = min(start + max_tokens, num_tokens_in_sentence)
                current_chunk = []
                current_token_count = 0
                continue

            if current_token_count + num_tokens_in_sentence > max_tokens:
                # Se il chunk corrente supera il limite, lo aggiungiamo e iniziamo un nuovo chunk
                chunks.append({"title": title, "text": " ".join(current_chunk), "url": url})
                current_chunk = []
                current_token_count = 0

            current_chunk.append(sentence)
            current_token_count += num_tokens_in_sentence

        # Aggiungi l'ultimo chunk se presente
        if current_chunk:
            chunks.append({"title": title, "text": " ".join(current_chunk), "url": url})

      except AssertionError :
        pass
    return chunks





In [33]:
all_chunks = chunk_text_using_stanza(all_data)

In [34]:
len(all_chunks)

4178

In [37]:
import pickle

pickle.dump(all_chunks, open('all_chunks.pkl', 'wb'))